In [48]:
import os
from tqdm import tqdm
import json
from xml.etree import ElementTree as ET

anno_path = "E:/Datasets/Extracted/IIIT-AR-13K_dataset/validation_xml"
output_annotation_file = "E:/Datasets/Extracted/IIIT-AR-13K_dataset/val.json"

train = range(1000)
val = range(500)

In [49]:
import re

def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower() 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

In [50]:
coco_data = {}
files = natural_sort(os.listdir(anno_path))    #get all files in the folder
cats = []

'''
for i in tqdm(range(len(files))):
    xml_path = anno_path + '/' + files[i]
    xml = ET.parse(xml_path)
    #print(xml['annotations'])
    
    root = xml.getroot()

    names = root.findall('object/name')
    for name in names:
        if name.text not in cats:
            cats.append(name.text)
cats = sorted(cats)
'''

cats = ['abstract', 'author', 'caption', 'date', 'equation', 'figure', 'footer', 'formula', 'list', 'logo', 'natural_image', 'paragraph', 'reference', 'section', 'signature', 'table', 'title']

list_dict = []
for i in range(len(cats)):
    tmp = {}
    tmp["supercategory"] = "none"
    tmp["id"] = i
    tmp["name"] = cats[i]
    list_dict.append(tmp)

coco_data["categories"] = list_dict
print(coco_data["categories"])           

[{'supercategory': 'none', 'id': 0, 'name': 'abstract'}, {'supercategory': 'none', 'id': 1, 'name': 'author'}, {'supercategory': 'none', 'id': 2, 'name': 'caption'}, {'supercategory': 'none', 'id': 3, 'name': 'date'}, {'supercategory': 'none', 'id': 4, 'name': 'equation'}, {'supercategory': 'none', 'id': 5, 'name': 'figure'}, {'supercategory': 'none', 'id': 6, 'name': 'footer'}, {'supercategory': 'none', 'id': 7, 'name': 'formula'}, {'supercategory': 'none', 'id': 8, 'name': 'list'}, {'supercategory': 'none', 'id': 9, 'name': 'logo'}, {'supercategory': 'none', 'id': 10, 'name': 'natural_image'}, {'supercategory': 'none', 'id': 11, 'name': 'paragraph'}, {'supercategory': 'none', 'id': 12, 'name': 'reference'}, {'supercategory': 'none', 'id': 13, 'name': 'section'}, {'supercategory': 'none', 'id': 14, 'name': 'signature'}, {'supercategory': 'none', 'id': 15, 'name': 'table'}, {'supercategory': 'none', 'id': 16, 'name': 'title'}]


In [51]:
coco_data["images"] = []
coco_data["annotations"] = []

ann_id = 0

for i in tqdm(val):
    
    xml_path = anno_path + '/' + files[i]
    xml = ET.parse(xml_path)
    
    root = xml.getroot()
    img_name = files[i].replace('xml','jpg')
    
    sizes = root.findall('size')
    w = sizes[0][0].text
    h = sizes[0][1].text
    
    images_tmp = {}
    images_tmp["file_name"] = img_name
    images_tmp["height"] = h
    images_tmp["width"] = w
    images_tmp["id"] = i
    coco_data["images"].append(images_tmp)
    
    objs = root.findall('object')
    for obj in objs:
        
        label = obj.find('name').text
        x1 = int(obj.find('bndbox/xmin').text)
        y1 = int(obj.find('bndbox/ymin').text)
        x2 = int(obj.find('bndbox/xmax').text)
        y2 = int(obj.find('bndbox/ymax').text)
    
        annotations_tmp = {}
        annotations_tmp["id"] = ann_id
        ann_id += 1
        annotations_tmp["image_id"] = i
        annotations_tmp["segmentation"] = []
        annotations_tmp["ignore"] = 0
        annotations_tmp["area"] = (x2-x1)*(y2-y1)
        annotations_tmp["iscrowd"] = 0
        annotations_tmp["bbox"] = [x1, y1, x2-x1, y2-y1]
        annotations_tmp["category_id"] = cats.index(label)

        coco_data["annotations"].append(annotations_tmp)

100%|██████████████████████████████████████| 500/500 [00:00<00:00, 4901.69it/s]


In [52]:
outfile =  open(output_annotation_file, 'w')
json_str = json.dumps(coco_data, indent=4)
outfile.write(json_str)
outfile.close()